In [ ]:
#https://www.usamoneylenders.com/payday-loans--credit/

#*Not Working Yet*

In [ ]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from bs4 import BeautifulSoup
import time
import csv


driver = webdriver.Chrome()

# Extract office details 
def tm_offices(lender_url):

    driver.get(lender_url)
    print(f"Accessing page: {lender_url}")
    
    # Wait for the user to solve CAPTCHA manually, if prompted
    input("Solve CAPTCHA if prompted, then press Enter to continue...")
    time.sleep(10)

    # Parse the rendered page with BeautifulSoup
    soup = BeautifulSoup(driver.page_source, 'html.parser')

    # office data
    office_data = []

    # Extracting office info *need to work on this section*
    for office in soup.find_all('div', class_='office-details'):
        office_name = office.find('h3').text.strip() if office.find('h3') else 'N/A'
        office_address = office.find('p', class_='address').text.strip() if office.find('p', class_='address') else 'N/A'
        office_city = office.find('p', class_='city').text.strip() if office.find('p', class_='city') else 'N/A'
        office_zip = office.find('p', class_='zip').text.strip() if office.find('p', class_='zip') else 'N/A'
        office_state = office.find('p', class_='state').text.strip() if office.find('p', class_='state') else 'N/A'
        office_phone = office.find('span', class_='phone').text.strip() if office.find('span', class_='phone') else 'N/A'

        
        # Add the extracted data to the office_data list
        office_data.append([office_name, office_address, office_city, office_zip, office_state, office_phone])

    return office_data

# Main scraping loop
lender_url = "https://www.usamoneylenders.com/payday-loans--credit/"

page_number = 1
all_office_data = []

while True:
    print(f"Scraping page {page_number}...")
    
    # scrape data from the current page
    office_data = tm_offices(lender_url)
    if not office_data:
        # Stop if no more data
        break  

    # Add the office data to the final list
    all_office_data.extend(office_data)

    # Check if there is a "Next" button 
    try:
        next_button = driver.find_element(By.ID, 'DataTables_Table_O_next')
        # Click "Next" if the button is enabled
        if next_button.is_enabled(): 
            next_button.click()
            time.sleep(10)  
            page_number += 1
        else:
            # Exit if no "Next" button or it's disabled
            break  
    except Exception as e:
        break

# Save data to a CSV file
with open('office_data.csv', mode='w', newline='') as file:
    writer = csv.writer(file)
     # Write header
    writer.writerow(['Office Name', 'Address', 'Phone Number', 'City', 'state',])  
    # Write all rows of office data
    writer.writerows(all_office_data) 

print(f"Total number of offices: {len(all_office_data)}")


driver.quit()
